In [1]:
from robyn import transformation
from robyn import data, inputs, checks, model, outputs, json, plots, response, allocator ## Manual, Added manually
import numpy as np
import pandas as pd

/Users/sandeepmane/.virtualenvs/r-reticulate/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
## Check simulated dataset or load your own dataset
dt_simulated_weekly = data.dt_simulated_weekly()
print(dt_simulated_weekly.head())

## Check holidays from Prophet
# 59 countries included. If your country is not included, please manually add it.
# Tipp: any events can be added into this table, school break, events etc.
dt_prophet_holidays = data.dt_prophet_holidays()
print(dt_prophet_holidays.head())


# Directory where you want to export results to (will create new folders)
robyn_directory = "~/Desktop"

         DATE       revenue          tv_S     ooh_S       print_S  \
0  2015-11-23  2.754372e+06  22358.346667       0.0  12728.488889   
1  2015-11-30  2.584277e+06  28613.453333       0.0      0.000000   
2  2015-12-07  2.547387e+06      0.000000  132278.4    453.866667   
3  2015-12-14  2.875220e+06  83450.306667       0.0  17680.000000   
4  2015-12-21  2.215953e+06      0.000000  277336.0      0.000000   

     facebook_I  search_clicks_P     search_S  competitor_sales_B  \
0  2.430128e+07         0.000000     0.000000             8125009   
1  5.527033e+06      9837.238486  4133.333333             7901549   
2  1.665159e+07     12044.119653  3786.666667             8300197   
3  1.054977e+07     12268.070319  4253.333333             8122883   
4  2.934090e+06      9467.248023  3613.333333             7105985   

    facebook_S events    newsletter  
0  7607.132915     na  19401.653846  
1  1141.952450     na  14791.000000  
2  4256.375378     na  14544.000000  
3  2800.490677    

In [3]:
input_collect = inputs.robyn_inputs(
    dt_input=dt_simulated_weekly,
    dt_holidays=dt_prophet_holidays,
    date_var="DATE",
    dep_var="revenue",
    dep_var_type="revenue",
    prophet_vars=["trend", "season", "holiday"], ## Manually added list
    prophet_country="DE",
    context_vars=["competitor_sales_B", "events"],
    paid_media_spends=["tv_S", "ooh_S", "print_S", "facebook_S", "search_S"],
    paid_media_vars=["tv_S", "ooh_S", "print_S", "facebook_I", "search_clicks_P"],
    organic_vars=["newsletter"], ## Manually converted to list, since R behaves single variable as list as well.
    window_start="2016-01-01",
    window_end="2018-12-31",
    adstock="geometric"
)

# Print input collection
print(input_collect)

{'date_var': 'DATE', 'dayInterval': 7, 'intervalType': 'week', 'dt_input':           DATE       revenue          tv_S          ooh_S       print_S  \
0   2015-11-23  2.754372e+06  22358.346667       0.000000  12728.488889   
1   2015-11-30  2.584277e+06  28613.453333       0.000000      0.000000   
2   2015-12-07  2.547387e+06      0.000000  132278.400000    453.866667   
3   2015-12-14  2.875220e+06  83450.306667       0.000000  17680.000000   
4   2015-12-21  2.215953e+06      0.000000  277336.000000      0.000000   
..         ...           ...           ...            ...           ...   
203 2019-10-14  2.456240e+06      0.000000   32230.933333  20496.488889   
204 2019-10-21  2.182825e+06  20655.680000       0.000000      0.000000   
205 2019-10-28  2.377707e+06   2931.000000    7516.800000   2330.133333   
206 2019-11-04  2.732825e+06   2993.733333       0.000000   3206.844444   
207 2019-11-11  2.767788e+06      0.000000       0.000000      0.000000   

       facebook_I  searc

In [5]:
hyper_names = inputs.hyper_names(adstock=input_collect['robyn_inputs']['adstock'], all_media=input_collect['robyn_inputs']['all_media'])

In [6]:
transformation.plot_adstock(plot = True)

(<Figure Size: (640 x 480)>, <Figure Size: (640 x 480)>)

In [7]:
transformation.plot_saturation(plot = True)

TypeError: theme_gray.__init__() got an unexpected keyword argument 'background'

In [8]:
checks.hyper_limits()

,thetas,alphas,gammas,shapes,scales
0,>=0,>0,>0,>=0,>=0
1,<1,<10,<=1,<20,<=1


In [9]:
facebook_S_alphas = np.array([0.5, 3])
facebook_S_gammas = np.array([0.3, 1])
facebook_S_thetas = np.array([0, 0.3])
print_S_alphas = np.array([0.5, 3])
print_S_gammas = np.array([0.3, 1])
print_S_thetas = np.array([0.1, 0.4])
tv_S_alphas = np.array([0.5, 3])
tv_S_gammas = np.array([0.3, 1])
tv_S_thetas = np.array([0.3, 0.8])
search_S_alphas = np.array([0.5, 3])
search_S_gammas = np.array([0.3, 1])
search_S_thetas = np.array([0, 0.3])
ooh_S_alphas = np.array([0.5, 3])
ooh_S_gammas = np.array([0.3, 1])
ooh_S_thetas = np.array([0.1, 0.4])
newsletter_alphas = np.array([0.5, 3])
newsletter_gammas = np.array([0.3, 1])
newsletter_thetas = np.array([0.1, 0.4])
train_size = np.array([0.5, 0.8])

hyperparameters = pd.DataFrame({
        'facebook_S_alphas': facebook_S_alphas,
        'facebook_S_gammas': facebook_S_gammas,
        'facebook_S_thetas': facebook_S_thetas,
        'print_S_alphas': print_S_alphas,
        'print_S_gammas': print_S_gammas,
        'print_S_thetas': print_S_thetas,
        'tv_S_alphas': tv_S_alphas,
        'tv_S_gammas': tv_S_gammas,
        'tv_S_thetas': tv_S_thetas,
        'search_S_alphas': search_S_alphas,
        'search_S_gammas': search_S_gammas,
        'search_S_thetas': search_S_thetas,
        'ooh_S_alphas': ooh_S_alphas,
        'ooh_S_gammas': ooh_S_gammas,
        'ooh_S_thetas': ooh_S_thetas,
        'newsletter_alphas': newsletter_alphas,
        'newsletter_gammas': newsletter_gammas,
        'newsletter_thetas': newsletter_thetas,
        'train_size': train_size
    })

In [10]:
calibration_input = pd.DataFrame({
  # channel name must in paid_media_vars
  "channel": ["facebook_S",  "tv_S", "facebook_S+search_S", "newsletter"],
  # liftStartDate must be within input data range
  "liftStartDate": pd.to_datetime(["2018-05-01", "2018-04-03", "2018-07-01", "2017-12-01"]),
  # liftEndDate must be within input data range
  "liftEndDate": pd.to_datetime(["2018-06-10", "2018-06-03", "2018-07-20", "2017-12-31"]),
  # Provided value must be tested on same campaign level in model and same metric as dep_var_type
  "liftAbs": [400000, 300000, 700000, 200],
  # Spend within experiment: should match within a 10% error your spend on date range for each channel from dt_input
  "spend": [421000, 7100, 350000, 0],
  # Confidence: if frequentist experiment, you may use 1 - pvalue
  "confidence": [0.85, 0.8, 0.99, 0.95],
  # KPI measured: must match your dep_var
  "metric": ["revenue", "revenue", "revenue", "revenue"],
  # Either "immediate" or "total". For experimental inputs like Facebook Lift, "immediate" is recommended.
  "calibration_scope": ["immediate", "immediate", "immediate", "immediate"]
})

test = calibration_input["channel"]

In [11]:
input_collect = inputs.robyn_inputs(
    InputCollect = input_collect['robyn_inputs'],
    hyperparameters = hyperparameters,
    calibration_input = calibration_input
)


>> Running Robyn feature engineering...


22:46:32 - cmdstanpy - INFO - Chain [1] start processing
22:46:33 - cmdstanpy - INFO - Chain [1] done processing
